# Bloch-Torrey Equation

## Introduction

Here we solve the Bloch-Torrey equation on a unit square, with the diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$ all given as a generic functions.
The strong form of the Bloch-Torrey equation is given by

\begin{align}
    \frac{\partial u_x}{\partial t} &= \nabla \cdot (D \nabla u_x) - R u_x + \omega u_y  \quad x \in \Omega\\
    \frac{\partial u_y}{\partial t} &= \nabla \cdot (D \nabla u_y) - R u_y - \omega u_x  \quad x \in \Omega,
\end{align}

where $\vec{u}=[u_x,u_y]$ is the transverse magnetization, and $\Omega$ the domain.

We will consider homogeneous Neumann boundary conditions such that

\begin{align}
    \nabla \vec{u}(x) \cdot \hat{n} &= 0  \quad x \in \partial \Omega\\
\end{align}

where $\partial \Omega$ denotes the boundary of $\Omega$. The initial condition is given generically as

\begin{equation}
    \vec{u}(x,t=0) = \vec{u}_0 (x)  \quad x \in \Omega
\end{equation}


The resulting weak form is given by
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    &= -\int_{\Omega}
    -\vec{v} \cdot \nabla \cdot ( D \, \nabla \vec{u} ) +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega \\
    &= -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega + 
    \int_{\partial\Omega} \vec{v} \cdot (D\nabla\vec{u} \cdot \hat{n}) \, d\Gamma,
\end{align}
where $\vec{v}$ is a suitable test function.

In this notebook, we will assume homogeneous Neumann boundary conditions on all boundaries by taking $D\nabla\vec{u} \cdot \hat{n} = 0$. Therefore, the final weak form is simply
\begin{align}
    \int_{\Omega} \vec{v} \cdot \vec{u}_t \, d\Omega
    = -\int_{\Omega}
    D \, \nabla \vec{v} : \nabla \vec{u} +
    R \, \vec{v} \cdot \vec{u} -
    \omega \, \vec{v} \times \vec{u}
    \, d\Omega
\end{align}

Note that, in two dimensions, the cross product is simply a scalar. However, `Tensors.jl` defines the two dimensional cross product by first extending the 2D vectors into 3D. Below, we use the symbol $\boxtimes$ to denote the scalar version, which is the same as taking the third component of the vector version

## Commented Program

Now we solve the problem in JuAFEM. What follows is a program spliced with comments.

First we load the JuAFEM package.

In [485]:
HOME = "/home/coopar7/Documents/code/"
#HOME = "/home/jon/Documents/UBCMRI/"
cd(HOME * "BlochTorreyExperiments-master/")

using Traceur
using BenchmarkTools
using StaticArrays
using JuAFEM
using JuAFEM: vertices, faces, edges
using MATLAB
using LinearMaps
using DifferentialEquations
using Expokit
using Optim
using Roots
using Distributions
using ApproxFun
#using Plots
using ForwardDiff
using ReverseDiff
using IterTools

include("Experiments/MyelinWaterOrientation/Geometry/geometry_utils.jl")
include("Experiments/MyelinWaterOrientation/Geometry/circle_packing.jl")
include("Experiments/MyelinWaterOrientation/Utils/mesh_utils.jl")
include("Experiments/MyelinWaterOrientation/Utils/normest1.jl")
include("Experiments/MyelinWaterOrientation/Utils/blochtorrey_utils.jl")
Revise.track("Experiments/MyelinWaterOrientation/Geometry/geometry_utils.jl")
Revise.track("Experiments/MyelinWaterOrientation/Geometry/circle_packing.jl")
Revise.track("Experiments/MyelinWaterOrientation/Utils/mesh_utils.jl")
Revise.track("Experiments/MyelinWaterOrientation/Utils/normest1.jl")
Revise.track("Experiments/MyelinWaterOrientation/Utils/blochtorrey_utils.jl")

using Normest1

**Pack circles**: Pack circles with a specified packing density $\eta$

In [486]:
btparams = BlochTorreyParameters();
freqparams = FreqMapParams(btparams);

In [487]:
# α or k == R_shape, θ == R_scale
R_mu = 0.46 # Axon mean radius [um] ; this is taken to be outer radius
R_shape = 5.7 # Axon radius shape parameter for Gamma distribution (Xu)
R_scale = R_mu / R_shape # Axon radius scale parameter [um]
R_σ = sqrt(R_shape)*R_scale; # Axon radius variance

const Dim = 2
Ncircles = 100
rs = rand(Gamma(R_shape, R_scale), Ncircles);
os = initialize_origins(rs);

η = 0.7 # goal packing density
ϵ = 0.1*R_mu # overlap occurs when distance between circle edges is ≤ ϵ
α = 0.0 # density penalty weight
β = 1e-5 # mutual distance penalty weight
λ = 1.0 # overlap penalty weight (or lagrange multiplier for constrained version)
w = [α, β, λ]; # vector of weights

In [488]:
revise()

In [489]:
@time outer_circles, opt_result = pack_circles(rs;
    autodiff = true,
    reversemode = false,
    initial_origins = os,
    goaldensity = η,
    distancescale = R_mu,
    weights = w,
    epsilon = ϵ);

In [490]:
estimate_density(outer_circles)

0.7000000000000002

In [491]:
is_any_overlapping(outer_circles)

false

In [492]:
revise()

 15.066821 seconds (4.60 M allocations: 184.934 MiB)


In [495]:
g_ratio = 0.5 #0.8370
#rect_bdry = bounding_box(outer_circles)
rect_bdry = scale_shape(inscribed_square(crude_bounding_circle(outer_circles)), 0.75)
#rect_bdry = scale_shape(bounding_box(outer_circles), 1.05)

outer_circles = filter(c -> !is_outside(c, rect_bdry), outer_circles)
inner_circles = scale_shape.(outer_circles, g_ratio);
Ncircles = length(outer_circles)

74

In [557]:
revise()

In [561]:
#Nmin = 50; # points for average circle
#h0 = 2pi*R_mu/Nmin; # approximate scale
h0 = 0.3*R_mu*(1-g_ratio) # fraction of size of average torus width
eta = 3.0; # approx ratio between largest/smallest edges

@time grid = square_mesh_with_tori(rect_bdry, inner_circles, outer_circles, h0, eta, isunion=true);
# @time grid, subgrids = square_mesh_with_circles(rect_bdry, outer_circles, h0, eta, isunion=false);

In [562]:
mxcall(:figure,0); mxcall(:hold,0,"on"); mxplot(grid); sleep(0.1);

 53.115335 seconds (327.64 k allocations: 10.135 MiB)


In [563]:
exteriorgrid, torigrids, interiorgrids = get_tori_subgrids(grid, rect_bdry, inner_circles, outer_circles);

In [564]:
all_tori = form_subgrid(grid, getcellset(grid, "tori"), getnodeset(grid, "tori"), getfaceset(grid, "boundary"))
all_int = form_subgrid(grid, getcellset(grid, "interior"), getnodeset(grid, "interior"), getfaceset(grid, "boundary"))
mxcall(:figure,0); mxcall(:hold,0,"on"); mxplot(exteriorgrid); sleep(0.1);
mxcall(:figure,0); mxcall(:hold,0,"on"); mxplot(all_tori); sleep(0.1);
mxcall(:figure,0); mxcall(:hold,0,"on"); mxplot(all_int); sleep(0.1);

In [565]:
domain = MyelinDomain(grid, outer_circles, inner_circles, rect_bdry, exteriorgrid, torigrids, interiorgrids;
    quadorder = 1, funcinterporder = 1);

LoadError: [91mBoundsError: attempt to access 1-element Array{Int64,1} at index [2][39m

In [566]:
doassemble!(domain, btparams);

In [387]:
factorize!(domain);

In [372]:
using Cubature

In [419]:
lb, ub = Vector(minimum(rect_bdry)), Vector(maximum(rect_bdry))
(val,err) = hcubature(1, (x,v) -> (y = exp.(2*x.^2/10); v[1] = dot(y,y)), lb, ub)

([75.8078], [7.51604e-7])

In [425]:
# U = interpolate(x->Vec{2}(exp.(2x⊙x/10)), domain);
U = interpolate(Vec{2}((0.0,1.0)), domain);
S = integrate(U, domain)

2-element Tensors.Tensor{1,2,Float64,2}:
  0.0   
 15.7765

In [414]:
norm(U, domain)

8.70773349520356

### Diffusion coefficient $D(x)$, relaxation rate $R(x)$, and resonance frequency $\omega(x)$

These functions are defined within `doassemble!`

### Trial and test functions
A `CellValues` facilitates the process of evaluating values and gradients of
test and trial functions (among other things). Since the problem
is a scalar problem we will use a `CellScalarValues` object. To define
this we need to specify an interpolation space for the shape functions.
We use Lagrange functions (both for interpolating the function and the geometry)
based on the reference "cube". We also define a quadrature rule based on the
same reference cube. We combine the interpolation and the quadrature rule
to a `CellScalarValues` object.

### Degrees of freedom
Next we need to define a `DofHandler`, which will take care of numbering
and distribution of degrees of freedom for our approximated fields.
We create the `DofHandler` and then add a single field called `u`.
Lastly we `close!` the `DofHandler`, it is now that the dofs are distributed
for all the elements.

Now that we have distributed all our dofs we can create our tangent matrix,
using `create_sparsity_pattern`. This function returns a sparse matrix
with the correct elements stored.

We can inspect the pattern using the `spy` function from `UnicodePlots.jl`.
By default the stored values are set to $0$, so we first need to
fill the stored values, e.g. `K.nzval` with something meaningful.

In [ ]:
#using UnicodePlots
#fill!(K.nzval, 1.0)
#spy(K; height = 25)

### Boundary conditions
In JuAFEM constraints like Dirichlet boundary conditions are handled by a `ConstraintHandler`. However, here we will have no need to directly enforce boundary conditions, since Neumann boundary conditions have already been applied in the derivation of the weak form.

### Assembling the linear system
Now we have all the pieces needed to assemble the linear system, $K u = f$.
We define a function, `doassemble` to do the assembly, which takes our `cellvalues`,
the sparse matrix and our DofHandler as input arguments. The function returns the
assembled stiffness matrix, and the force vector.

### Solution of the differential equation system
The last step is to solve the system. First we call `doassemble`
to obtain the global stiffness matrix `K` and force vector `f`.
Then, to account for the boundary conditions, we use the `apply!` function.
This modifies elements in `K` and `f` respectively, such that
we can get the correct solution vector `u` by using `\`.

In [472]:
tspan = (0.0,40e-3);
u0 = Vec{2}((0.0, 1.0))
U0 = interpolate(u0, domain); # vector of vectors
#U = similar(U0)

In [463]:
for i in numsubdomains(domain):-1:1
    print("i = $i: ")
    
    subdomain = getsubdomain(domain, i)
    Amap = paraboliclinearmap(subdomain)
    U[i] = similar(U0[i])
        
    # Method 1: expmv! from Expokit.jl
    #@time Expokit.expmv!(U[i], tspan[end], Amap, U0[i]; tol=1e-4, norm=expmv_norm, m=30);
    
    # Method 2: direct ODE solution using DifferentialEquations.jl
    prob = ODEProblem((du,u,p,t)->A_mul_B!(du,p[1],u), U0[i], tspan, (Amap,));
    @time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-4, alg_hints=:stiff)
    U[i] = sol.u[end]
end

  1.332069 seconds (1.04 M allocations: 325.568 MiB, 11.94% gc time)
i = 21
  0.007311 seconds (637 allocations: 2.484 MiB)
i = 20
  3.644639 seconds (401.01 k allocations: 1.067 GiB, 27.50% gc time)
i = 19
  2.099000 seconds (419.61 k allocations: 838.806 MiB, 10.10% gc time)
i = 18
  0.345736 seconds (197.85 k allocations: 128.953 MiB, 18.81% gc time)
i = 17
  0.499486 seconds (323.81 k allocations: 196.626 MiB, 15.71% gc time)
i = 16
  1.303666 seconds (78.35 k allocations: 199.384 MiB, 5.75% gc time)
i = 15
  1.543530 seconds (235.62 k allocations: 579.391 MiB, 9.90% gc time)
i = 14
  3.925535 seconds (725.01 k allocations: 1.498 GiB, 9.33% gc time)
i = 13
  5.443864 seconds (162.40 k allocations: 722.565 MiB, 3.76% gc time)
i = 12
  8.370402 seconds (10.14 M allocations: 3.884 GiB, 11.63% gc time)
i = 11
 11.932582 seconds (1.65 M allocations: 4.978 GiB, 9.19% gc time)
i = 10
 13.361518 seconds (3.02 M allocations: 5.909 GiB, 10.30% gc time)
i = 9
  6.871944 seconds (2.10 M alloca


[CVODES ERROR]  CVode
  At t = 0.0355854 and h = 1.0882e-08, the corrector convergence test failed repeatedly or with |h| = hmin.



19.372542 seconds (21.31 M allocations: 9.043 GiB, 11.62% gc time)
i = 6
  1.330600 seconds (136.52 k allocations: 509.477 MiB, 10.84% gc time)
i = 5
  0.347714 seconds (82.69 k allocations: 143.561 MiB, 20.34% gc time)
i = 4
  6.411885 seconds (2.03 M allocations: 2.893 GiB, 11.07% gc time)
i = 3
  2.380921 seconds (119.96 k allocations: 800.125 MiB, 8.83% gc time)
i = 2
464.932885 seconds (3.12 M allocations: 50.740 GiB, 2.25% gc time)
i = 1


In [476]:
u0 * area(domain.domainboundary)

2-element Tensors.Tensor{1,2,Float64,2}:
  0.0   
 15.7765

In [477]:
integrate(U, domain)

2-element Tensors.Tensor{1,2,Float64,2}:
 -0.846457
  3.32462 

In [481]:
exp(-tspan[end]*btparams[:R2_lp])

0.5299775483586843

In [ ]:
prob = ODEProblem((du,u,p,t)->A_mul_B!(du,p[1],u), u0, tspan, (Amap,));

In [ ]:
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=100); # penelope: 17.42s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=50); # penelope: 30.09s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-4, norm=expmv_norm, m=10); # penelope: 103.5s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-8, norm=expmv_norm); # penelope: 53.2s
#@time Expokit.expmv!(u, tspan[end], Amap, u0; tol=1e-6, norm=expmv_norm); # penelope: 44.4s

In [ ]:
#@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-8, alg_hints=:stiff); # penelope: 90.21s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:GMRES); saveat=tspan, reltol=1e-4, alg_hints=:stiff); # penelope: 33.44s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:BCG); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 53.66s
#@time sol = solve(prob, CVODE_BDF(linear_solver=:TFQMR); saveat=tspan, reltol=1e-4, alg_hints=:stiff) # penelope: 18.99s but low accuracy

In [ ]:
#prob_Ku = ODEProblem(K_mul_u!, u0, tspan, (K,), mass_matrix=M);
#@time sol_Ku = solve(prob_Ku, Rosenbrock23(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF
#@time sol_Ku = solve(prob_Ku, Rodas4(), saveat=tspan, reltol=1e-4, alg_hints=:stiff) #DNF

In [ ]:
@show norm(sol.u[end] - u)/maximum(abs,u);
@show maximum(sol.u[end] - u)/maximum(abs,u);

### Exporting to VTK
To visualize the result we export the grid and our field `u`
to a VTK-file, which can be viewed in e.g. [ParaView](https://www.paraview.org/).

In [ ]:
vtk_grid("bloch_torrey_equation", dh) do vtk
    vtk_point_data(vtk, dh, u)
end

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*